# What to do now :

Make a Portfolio using the strategy given. Test the strategy on a historical dataset. Evaluate the performance of the strategy using metrics such as Sharpe Ratio, etc. Make graphical representation.

Note: We are using data for a day.


In [11]:
import pandas as pd
import numpy as np

In [12]:
# No Need to download data again
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
# print(t)
t=pd.read_csv("Tickers.csv")
t=t.iloc[:,1:2]
t.reset_index(drop=True, inplace=True)
t=t.values
tickers=[]
for stock_group in t:
    for stock_name in stock_group:
        tickers.append(stock_name)
    
tickers


['A',
 'AAPL',
 'ABBV',
 'ABNB',
 'ABT',
 'ACGL',
 'ACN',
 'ADBE',
 'ADI',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AES',
 'AFL',
 'AIG',
 'AIZ',
 'AJG',
 'AKAM',
 'ALB',
 'ALGN',
 'ALL',
 'ALLE',
 'AMAT',
 'AMCR',
 'AMD',
 'AME',
 'AMGN',
 'AMP',
 'AMT',
 'AMZN',
 'ANET',
 'ANSS',
 'AON',
 'AOS',
 'APA',
 'APD',
 'APH',
 'APO',
 'APTV',
 'ARE',
 'ATO',
 'AVB',
 'AVGO',
 'AVY',
 'AWK',
 'AXON',
 'AXP',
 'AZO',
 'BA',
 'BAC',
 'BALL',
 'BAX',
 'BBY',
 'BDX',
 'BEN',
 'BF.B',
 'BG',
 'BIIB',
 'BK',
 'BKNG',
 'BKR',
 'BLDR',
 'BLK',
 'BMY',
 'BR',
 'BRK.B',
 'BRO',
 'BSX',
 'BX',
 'BXP',
 'C',
 'CAG',
 'CAH',
 'CARR',
 'CAT',
 'CB',
 'CBOE',
 'CBRE',
 'CCI',
 'CCL',
 'CDNS',
 'CDW',
 'CEG',
 'CF',
 'CFG',
 'CHD',
 'CHRW',
 'CHTR',
 'CI',
 'CINF',
 'CL',
 'CLX',
 'CMCSA',
 'CME',
 'CMG',
 'CMI',
 'CMS',
 'CNC',
 'CNP',
 'COF',
 'COIN',
 'COO',
 'COP',
 'COR',
 'COST',
 'CPAY',
 'CPB',
 'CPRT',
 'CPT',
 'CRL',
 'CRM',
 'CRWD',
 'CSCO',
 'CSGP',
 'CSX',
 'CTAS',
 'CTRA',
 'CTSH',
 'CTVA',


## STRATEGIES ------------------------------------------------------------------------ START

In [13]:
from scipy import stats
                                
def strat_per(stock, max_weight=0.08, momentum_days=20, reversal_days=5):
    """
    Simpler strategy combining short-term reversal with medium-term momentum
    """
    dfs = []
    n = None

    for ticker in stock:
        df = pd.read_csv(f"{ticker}.csv")
        df.reset_index(drop=True, inplace=True)
        df['Close'] = pd.to_numeric(df['Close'])
        dfs.append(df)
        if n is None:
            n = len(df)

    portfolio = np.zeros((len(stock), n))

    for i, df in enumerate(dfs):
        returns = df['Close'].pct_change()
        
        # Medium-term momentum
        momentum = returns.rolling(window=momentum_days).mean()
        
        # Short-term reversal
        short_term_return = returns.rolling(window=reversal_days).mean()
        reversal = -short_term_return  # Opposite of recent performance
        
        # Combine: follow medium-term trend but fade short-term moves
        signal = 0.7 * momentum + 0.3 * reversal # Try changing parameters. Might get some better results 
        
        # Volatility adjustment
        vol = returns.rolling(window=20).std()
        vol_adjusted_signal = signal / (vol + 1e-8) # Avoid division by zero. Important.
        
        portfolio[i, :] = vol_adjusted_signal.shift(1).fillna(0).to_numpy()

    abs_sum = np.sum(np.abs(portfolio), axis=0, keepdims=True)
    abs_sum[abs_sum == 0] = 1.0 # Prevents division by 0
    normalized_portfolio = portfolio / abs_sum # Normalising the portfolio to have a sum of 1
    clipped_portfolio = np.clip(normalized_portfolio, -max_weight, max_weight) # Clip to prevent over-weighting
    market_neutral_portfolio = clipped_portfolio - clipped_portfolio.mean(axis=0, keepdims=True) # Subtract the mean to make it market neutral
    
    return market_neutral_portfolio


## Assignment 
Try creating your own strategies. Just remember, input can be anything ,but , output should be a portfolio with rows as stocks and columns as dates. See example output if you don't understand.
Don't worry if this doesn't correspond to the other backtester. This is because the normalization you might be doing here is different than than how the other does that.

P.S. Before you backtest , make sure to complete the backtester : backtest_portfolio () function. 

In [9]:
def your_strat_1(stock, max_weight=0.3, momentum_days=50, reversal_days=20):
    """
    Simpler strategy combining short-term reversal with medium-term momentum
    """
    dfs = []
    n = None

    for ticker in stock:
        df = pd.read_csv(f"{ticker}.csv")
        df.reset_index(drop=True, inplace=True)
        df['Close'] = pd.to_numeric(df['Close'])
        dfs.append(df)
        if n is None:
            n = len(df)

    portfolio = np.zeros((len(stock), n))

    for i, df in enumerate(dfs):
        returns = df['Close'].pct_change()
        
        # Medium-term momentum
        momentum = returns.rolling(window=momentum_days).mean()
        
        # Short-term reversal
        short_term_return = returns.rolling(window=reversal_days).mean()
        reversal = -short_term_return  # Opposite of recent performance
        
        # Combine: follow medium-term trend but fade short-term moves, now its more short term focused
        signal = 0.7 * momentum + 0.3 * reversal # Try changing parameters. Might get some better results 
        
        # Volatility adjustment
        vol = returns.rolling(window=20).std()
        vol_adjusted_signal = signal / (vol + 1e-8) # Avoid division by zero. Important.
        
        portfolio[i, :] = vol_adjusted_signal.shift(1).fillna(0).to_numpy()

    abs_sum = np.sum(np.abs(portfolio), axis=0, keepdims=True)
    abs_sum[abs_sum == 0] = 1.0 # Prevents division by 0
    normalized_portfolio = portfolio / abs_sum # Normalising the portfolio to have a sum of 1
    clipped_portfolio = np.clip(normalized_portfolio, -max_weight, max_weight) # Clip to prevent over-weighting
    market_neutral_portfolio = clipped_portfolio - clipped_portfolio.mean(axis=0, keepdims=True) # Subtract the mean to make it market neutral
    
    return market_neutral_portfolio
    
    return -1

In [16]:
def your_strat_2(stocks, max_weight=0.3, momentum_days=120, reversal_days=20):
    dfs = []
    n = None
    for ticker in stocks:
        df = pd.read_csv(f"{ticker}.csv")
        df.reset_index(drop=True, inplace=True)
        df['Close'] = pd.to_numeric(df['Close'])
        dfs.append(df)
        if n is None:
            n = len(df)
    portfolio = np.zeros((len(stocks), n))
    
    for i, df in enumerate(dfs):
        df['Close'] = df['Close'].fillna(method='ffill')
        returns = df['Close'].pct_change()

    
    # Calculate SMA over a rolling window (momentum_days + reversal_days)
    window = momentum_days + reversal_days
    sma = returns.rolling(window=window).mean()

    momentum = 0
    reversal = 0

    # Iterate over valid indices where SMA is available
    for idx in range(window - 1, len(returns)):
        current_return = returns.iloc[idx]
        current_sma = sma.iloc[idx]

        if pd.isna(current_sma) or pd.isna(current_return):
            continue

        if current_return > current_sma:
            # Medium-term momentum
            momentum += current_return
        else:
            # Short-term reversal
            reversal += -current_return

    # Combine signals: follow medium-term trend but fade short-term moves
    signal = 0.3 * momentum + 0.7 * reversal # Try changing parameters. Might get some better results
    # Volatility adjustment
    vol = returns.rolling(window=10).std()
    vol_adjusted_signal = signal / (vol + 1e-8)
    portfolio[i, :] = vol_adjusted_signal.shift(1).fillna(0).to_numpy()
    abs_sum = np.sum(np.abs(portfolio), axis=0, keepdims=True)
    abs_sum[abs_sum == 0] = 1.0  # Prevents division by 0
    normalized_portfolio = portfolio / abs_sum  # Normalising the portfolio to have a sum of 1
    clipped_portfolio = np.clip(normalized_portfolio, -max_weight, max_weight)  # Clip to prevent over-weighting
    market_neutral_portfolio = clipped_portfolio - clipped_portfolio.mean(axis=0, keepdims=True)  # Subtract the mean to make it market neutral
    return market_neutral_portfolio 
    return -1

In [ ]:
def your_strat_3(stocks):
    
    return -1

## STRATEGIES ------------------------------------------------------ END

In [20]:
print("Testing the strat......\n")
portfolio = your_strat_2(tickers)
# print(portfolio.shape)
print(portfolio)  # Show first 10 elements
print(np.sum(portfolio))   # Sum of allocations : should be ideally 0 after Market Neutralisation

Testing the strat......



C:\Users\arnav\AppData\Local\Temp\ipykernel_9000\815244121.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Close'] = df['Close'].fillna(method='ffill')
C:\Users\arnav\AppData\Local\Temp\ipykernel_9000\815244121.py:15: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = df['Close'].pct_change()


[[ 0.          0.          0.         ... -0.00059642 -0.00059642
  -0.00059642]
 [ 0.          0.          0.         ... -0.00059642 -0.00059642
  -0.00059642]
 [ 0.          0.          0.         ... -0.00059642 -0.00059642
  -0.00059642]
 ...
 [ 0.          0.          0.         ... -0.00059642 -0.00059642
  -0.00059642]
 [ 0.          0.          0.         ... -0.00059642 -0.00059642
  -0.00059642]
 [ 0.          0.          0.         ...  0.29940358  0.29940358
   0.29940358]]
-1.1368683772161603e-13


In [19]:
print(portfolio.shape)

(503, 1006)


# Studying the metrics

The strategy was written in such a way that we didn't need to make a portfolio. 

All we had to do is just call the strat and the portfolio would take care of itself.

We have the portfolio, which is what the person's standing for the day is.

How much percentage of his portfolio is each stock in? So, basically we know the no. of stocks he owns for the day, given the portfolio.

So what do we need to do now ?

- As a day passes , we need to update the portfolio. ( We can also assume the portfolio to be constant for simplicity but we won't )

            PORTFOLIO_VAL_NEW  = PORTFOLIO_VAL_OLD + PnL
        
 We can also clip it for saftey.


- Calculate metrics as needed :
    
    - Total PnL

                    PnL = Summ ( ( Close_new- Close_old ) * Quantity ) for all stocks in the portfolio
    
    - Cumilitive Returns

                  Cumulative Return=( ∏ ( 1 + r_t ) ) − 1  : r_t daily return

    - Sharpe Ratio 

                    Sharpe Ratio = ( PnL - Risk_free ) / Volatility

        aka

                    Sharpe Ration = (Mean(Ret_daily) / Std(Ret_daily)) * sqrt(252)

    - Maximum Drawdown

                    Max Drawdown = max { ( V_peak - V_trough ) / V_peak } 
                    V :  portfolio value at time t = Sum ( weight of stock * Price )


    - Turnover

                    Turnover_at_t = Sum (( | weight_new - weight_old | )) : for all stocks in the portfolio

                    Turnover = Sum ( Turnover_at_t ) / days : for all days in the portfolio


    - Correlation : A metric introduced by me. It is the correlation between the stock and the portfolio. We'll measure it in time shift. 

                    Corr(  delta(close*volume,1) , PnL created by Portfolio  )  


- Plotting the metrics : ) 

Note : we have used a lot of assumptions while calculating the metrics and creating this backtester. For example : we have assumed that he player's influence in the market is negligible. 

In [43]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as py

def backtest_portfolio(portfolio, tickers, risk_free_rate=0.0451):
    n_stocks, n_days = portfolio.shape
    price_matrix = np.zeros((n_stocks, n_days))

    # Load price data
    for i, ticker in enumerate(tickers):
        df = pd.read_csv(f"{ticker}.csv")
        df = df.reset_index(drop=True)
        df['Close'] = pd.to_numeric(df['Close'])
        price_matrix[i, :] = df['Close'].to_numpy()

    # # Stock daily returns
    stock_returns = pd.DataFrame(price_matrix).pct_change(axis=1).fillna(0).values

    # print(stock_returns)

    # Portfolio returns
    daily_returns = np.nansum(portfolio * stock_returns, axis=0)

    # Cumulative Returns
    cumulative_returns =  3.14*(stock_returns + 1) - 1 # .................... Complete this and run

    # Sharpe Ratio
    excess_returns = daily_returns#- risk_free_rate / 252
    sharpe_ratio =  (daily_returns - risk_free_rate)/sum((stock_returns[i]-cumulative_returns)**2/(n_days-1) ) #..................... Complete this and run

    # Max Drawdown
    cumulative_curve = np.cumprod(1 + daily_returns)
    rolling_max = np.maximum.accumulate(cumulative_curve)
    drawdown = 1 - cumulative_curve / rolling_max
    max_drawdown = np.max(drawdown)

    # Turnover
    turnover = 0
    for i in range(1, n_stocks):  
        turnover_at_t = stock_returns[i] - stock_returns[i-1]
        turnover += np.sum(np.abs(turnover_at_t)) / n_days
    
    
    # Mean Correlation with individual stocks
    correlations = []
    for i in range(n_stocks):
        x = daily_returns
        y = stock_returns[i, :]
        mask = ~np.isnan(x) & ~np.isnan(y)
        if np.sum(mask) > 1:
            corr = np.corrcoef(x[mask], y[mask])[0, 1]
            correlations.append(corr)
        else:
            correlations.append(np.nan)
    mean_correlation = np.nanmean(correlations)

    results = {
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "Turnover": turnover,
        "Total Return": cumulative_returns[-1] * 100,
        "Mean Correlation (Portfolio vs Stocks)": mean_correlation,
        "Daily Returns": daily_returns,
        "Cumulative Returns": cumulative_returns
    }

    # PLotting 
    days = np.arange(n_days)

    # Create subplots layout: 3 rows for returns, drawdown, top stocks
    fig = make_subplots(
        rows=4, cols=1,
        subplot_titles=("Cumulative Returns", "Daily Returns", "Drawdown Curve", "Portfolio vs Top 5 Stocks"),
        vertical_spacing=0.1,
        shared_xaxes=True
    )

    # Cumulative Returns
    fig.add_trace(go.Scatter(x=days, y=cumulative_returns, name="Cumulative Return"), row=1, col=1)

    # Daily Returns
    fig.add_trace(go.Scatter(x=days, y=daily_returns, name="Daily Return"), row=2, col=1)

    # Drawdown
    fig.add_trace(go.Scatter(x=days, y=drawdown, name="Drawdown"), row=3, col=1)

    # Portfolio vs Top 5 correlated stocks
    fig.add_trace(go.Scatter(x=days, y=daily_returns, name="Portfolio"), row=4, col=1)
    top_5_idx = np.argsort(correlations)[-5:]

    # for i in top_5_idx:
    #     print(stock_returns[i])
    # print(top_5_idx)

    for i in top_5_idx:
        fig.add_trace(go.Scatter(x=days, y=stock_returns[i], name=f"{tickers[i]}"), row=4, col=1)

    # Layout
    fig.update_layout(
        height=1200,
        title_text="Portfolio Performance Summary",
        showlegend=True
    )

    # Save and open in browser
    py.plot(fig, filename="portfolio_summary.html")

    return results, correlations


In [53]:
results,corr = backtest_portfolio(portfolio, tickers)

for key, value in results.items():
    if key not in ['Daily Returns', 'Cumulative Returns']:
        if np.isscalar(value):
            print(f"{key}: {value:.4f}")
    else:
            print(f"{key}: {value}")

# print(corr)

counter =0
for x in corr:
    if x>0:
        counter+=1

print("No. of correlating stocks : "+ str(counter))

C:\Users\arnav\AppData\Local\Temp\ipykernel_19008\2879905147.py:19: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.

c:\Users\arnav\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3045: RuntimeWarning:

invalid value encountered in divide

c:\Users\arnav\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_function_base_impl.py:3046: RuntimeWarning:

invalid value encountered in divide



Max Drawdown: 0.2507
Turnover: 9.1674
Mean Correlation (Portfolio vs Stocks): -0.1803
Daily Returns: [ 0.          0.          0.         ...  0.00116013 -0.00114969
  0.00059741]
Cumulative Returns: [[2.14       2.08958426 2.14928238 ... 2.14022481 2.13887707 2.12865825]
 [2.14       2.10947306 2.16502035 ... 2.14162628 2.14699052 2.12296817]
 [2.14       2.11019506 2.16478042 ... 2.14528045 2.13736437 2.14446419]
 ...
 [2.14       2.13179231 2.12185353 ... 2.13250042 2.15296128 2.14180706]
 [2.14       2.10255814 2.1640362  ... 2.14330536 2.13499036 2.11696437]
 [2.14       2.14046947 2.11589265 ... 2.16248614 2.14414606 2.14334429]]
No. of correlating stocks : 9
